### Flask中请求处理流程

In [ ]:
def __call__(self, environ, start_response):
    """Shortcut for :attr:`wsgi_app`"""
    return self.wsgi_app(environ, start_response)

### Flask.wsgi_app方法
* self.request_context(environ): 打开请求上下文
* self.preprocess_request(): 分发请求前的预处理
* self.dispatch_request():分发请求
* self.make_response(rv)：生成响应
* self.process_response(response)：处理响应

In [ ]:
def wsgi_app(self, environ, start_response):
    """The actual WSGI application.  This is not implemented in
    `__call__` so that middlewares can be applied:

        app.wsgi_app = MyMiddleware(app.wsgi_app)

    :param environ: a WSGI environment
    :param start_response: a callable accepting a status code,
                           a list of headers and an optional
                           exception context to start the response
    """
    with self.request_context(environ):
        rv = self.preprocess_request()
        if rv is None:
            rv = self.dispatch_request()
        response = self.make_response(rv)
        response = self.process_response(response)
        return response(environ, start_response)

#### 打开请求上下文
self.request_context(environ) 直接返回 _RequestContext实例
_RequestContext类定义了_ _enter _ _ 和_ _exit_ _方法：这里使用到了python的上下文管理器，with语句开始运行时，会在上下文管理器对象上调用_ _enter_ _方法。with语句运行结束后，会在上下文管理器对象上调用_ _exit_ _方法。
因此with self.request_context(environ)将创建的请求上下文压入 _ request_ctx_stack中
#### 请求上下文
RequestContext对象的属性有:
* app
* url_adapter：负责路由查找
* request
* session
* g 

In [ ]:
def request_context(self, environ):
    return _RequestContext(self, environ)
    
class _RequestContext(object):
    """The request context contains all request relevant information.  It is
    created at the beginning of the request and pushed to the
    `_request_ctx_stack` and removed at the end of it.  It will create the
    URL adapter and request object for the WSGI environment provided.
    """

    def __init__(self, app, environ):
        self.app = app
        self.url_adapter = app.url_map.bind_to_environ(environ)
        self.request = app.request_class(environ)
        self.session = app.open_session(self.request)
        self.g = _RequestGlobals()
        self.flashes = None

    def __enter__(self):
        _request_ctx_stack.push(self)

    def __exit__(self, exc_type, exc_value, tb):
        # do not pop the request stack if we are in debug mode and an
        # exception happened.  This will allow the debugger to still
        # access the request object in the interactive shell.
        if tb is None or not self.app.debug:
            _request_ctx_stack.pop()

#### 分发请求
* self.match_request():匹配请求
* self.view_functions :调用视图函数

In [ ]:
def dispatch_request(self):
    """Does the request dispatching.  Matches the URL and returns the
    return value of the view or error handler.  This does not have to
    be a response object.  In order to convert the return value to a
    proper response object, call :func:`make_response`.
    """
    try:
        endpoint, values = self.match_request()
        return self.view_functions[endpoint](**values)
    except HTTPException, e:
        handler = self.error_handlers.get(e.code)
        if handler is None:
            return e
        return handler(e)
    except Exception, e:
        handler = self.error_handlers.get(500)
        if self.debug or handler is None:
            raise
        return handler(e)

#### 匹配请求
获取请求上下文，使用url_adapter进行匹配

In [ ]:
def match_request(self):
    """Matches the current request against the URL map and also
    stores the endpoint and view arguments on the request object
    is successful, otherwise the exception is stored.
    """
    rv = _request_ctx_stack.top.url_adapter.match()
    request.endpoint, request.view_args = rv
    return rv

#### 生成响应
把视图函数返回的结果包装成Response

In [ ]:
def make_response(self, rv):
    """Converts the return value from a view function to a real
    response object that is an instance of :attr:`response_class`.

    The following types are allowd for `rv`:

    ======================= ===========================================
    :attr:`response_class`  the object is returned unchanged
    :class:`str`            a response object is created with the
                            string as body
    :class:`unicode`        a response object is created with the
                            string encoded to utf-8 as body
    :class:`tuple`          the response object is created with the
                            contents of the tuple as arguments
    a WSGI function         the function is called as WSGI application
                            and buffered as response object
    ======================= ===========================================

    :param rv: the return value from the view function
    """
    if isinstance(rv, self.response_class):
        return rv
    if isinstance(rv, basestring):
        return self.response_class(rv)
    if isinstance(rv, tuple):
        return self.response_class(*rv)
    return self.response_class.force_type(rv, request.environ)

#### 处理响应结果


In [ ]:
def process_response(self, response):
    """Can be overridden in order to modify the response object
    before it's sent to the WSGI server.  By default this will
    call all the :meth:`after_request` decorated functions.

    :param response: a :attr:`response_class` object.
    :return: a new response object or the same, has to be an
             instance of :attr:`response_class`.
    """
    session = _request_ctx_stack.top.session
    if session is not None:
        self.save_session(session, response)
    for handler in self.after_request_funcs:
        response = handler(response)
    return response